In [1]:
from sklearn.metrics import precision_recall_curve

def evaluate_model(model, data_loader, device):
    """Evaluates the model and computes various metrics."""
    model.eval()
    all_true_labels = []
    all_predicted_labels = []
    all_probabilities = []
    correct = 0
    total = 0
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            #outputs = model(inputs.permute(0,2,1))
            loss = criterion(outputs.view(-1, 2), labels.view(-1))
            test_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=2) #Softmax to get probabilities
            threshold = 0.5325 # Example: Lower threshold
            predicted = (probabilities[:, :, 1] > threshold).long()
            #_, predicted = torch.max(outputs.data, 2)

            all_true_labels.extend(labels.view(-1).cpu().numpy())
            all_predicted_labels.extend(predicted.view(-1).cpu().numpy())
            all_probabilities.extend(probabilities.view(-1,2).cpu().numpy())
            total += labels.view(-1).size(0)
            correct += (predicted.view(-1) == labels.view(-1)).sum().item()
    
    test_loss /= len(test_loader)
    print(f"Test Loss: {test_loss:.4f}")
    # Calculate metrics
    accuracy=100 * correct / total
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_true_labels, all_predicted_labels, average="binary", zero_division=0
    )
    try:
        auc = roc_auc_score(all_true_labels, np.array(all_probabilities)[:, 1]) #Calculate AUC
    except ValueError:
        auc = np.nan
        print("Only one class present in y_true. ROC AUC score is not defined in that case.")

    probabilities_positive_class = np.array(all_probabilities)[:, 1]  # Probability of the positive class
    precision_curve, recall_curve, thresholds = precision_recall_curve(all_true_labels, probabilities_positive_class)
    f1_scores = 2 * (precision_curve * recall_curve) / (precision_curve + recall_curve + 1e-10)  # F1 for each threshold
    best_threshold = thresholds[np.argmax(f1_scores)]  # Optimal threshold
    best_f1_score = np.max(f1_scores)

    print(f"Optimal Threshold: {best_threshold:.4f}")
    print(f"Best F1-Score at Optimal Threshold: {best_f1_score:.4f}")
    
    return precision, recall, f1, auc, accuracy,best_f1_score,best_threshold,test_loss

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from early_stopping_pytorch import EarlyStopping
import itertools
import pickle

class SequenceLabelingRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.02):
        super(SequenceLabelingRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout, bidirectional=True, bias=True)
        self.fc = nn.Linear(hidden_size*2, output_size) # *2 because bidirectional

    def forward(self, x):
        out, _ = self.rnn(x)  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        out = self.fc(out)  # Output for every timestep
        return out

# Load data (assuming data is loaded as you provided)
with open('/scratch/da3245/datasets/timeseries_voltage/stratified_no_overlap.pkl', 'rb') as f:
    data = pickle.load(f)

X_train = data['X_train']
y_train = data['y_train']
X_val = data['X_val']
y_val = data['y_val']
X_test = data['X_test']
y_test = data['y_test']

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1)
X_val = torch.tensor(X_val, dtype=torch.float32).unsqueeze(-1)
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(-1)

y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the hyperparameter grid
hyperparameters = {
    'hidden_size': [32, 64, 128], 
    'num_layers': [1, 2, 3], 
    'dropout_rate': [0.02, 0.05, 0.1], 
    'lr': [0.001, 0.0005, 0.0001]
}

# Flatten all combinations of hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
param_combinations = list(itertools.product(*hyperparameters.values()))
best_f1_score = -1
best_params = None
best_model = None

# Iterate over all parameter combinations
for params in param_combinations:
    hidden_size, num_layers, dropout_rate, lr = params
    print(f"Training model with params: {params}")

    # Initialize model, criterion, and optimizer
    model = SequenceLabelingRNN(input_size=1, hidden_size=hidden_size, num_layers=num_layers, output_size=2, dropout=dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    early_stopping = EarlyStopping(patience=7, verbose=True)

    # Training loop
    num_epochs = 30
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.view(-1, 2), labels.view(-1))
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 1 == 0:
            # Validation
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs.view(-1, 2), labels.view(-1))
                    val_loss += loss.item()

            val_loss /= len(val_loader)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), 'best_model.pth')
                print("Saving best model...")
            else:
                early_stopping(val_loss, model)

            if early_stopping.early_stop:
                print("Early stopping triggered!")
                break

    # Evaluate the model with the current hyperparameters
    precision, recall, f1, auc, accuracy, best_f1_score, best_thresh, test_loss = evaluate_model(model, test_loader, device)
    print(f"Test F1-Score for params {params}: {best_f1_score:.4f}")

    # Update best parameters if current model is better
    if best_f1_score > best_f1_score:
        best_f1_score = best_f1_score
        best_params = params
        best_model = model

# After training all combinations, print the best hyperparameters
print(f"Best Hyperparameters: {best_params}")
print(f"Best F1-Score: {best_f1_score:.4f}")

Training model with params: (32, 1, 0.02, 0.001)


/ext3/miniforge3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.02 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/ext3/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Validation loss decreased (inf --> 0.007530).  Saving model ...
EarlyStopping counter: 1 out of 7
Saving best model...
Saving best model...
Validation loss decreased (0.007530 --> 0.006688).  Saving model ...
EarlyStopping counter: 1 out of 7
Saving best model...
Validation loss decreased (0.006688 --> 0.006646).  Saving model ...
EarlyStopping counter: 1 out of 7
EarlyStopping counter: 2 out of 7
Saving best model...
Saving best model...
Validation loss decreased (0.006646 --> 0.006358).  Saving model ...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
EarlyStopping counter: 1 out of 7
Saving best model...
Saving best model...
Validation loss decreased (0.006358 --> 0.006053).  Saving model ...
Test Loss: 0.0058
Optimal Threshold: 0.3561
Best F1-Score at Optimal T

/ext3/miniforge3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.02 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Validation loss decreased (inf --> 0.008765).  Saving model ...
Validation loss decreased (0.008765 --> 0.008218).  Saving model ...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Validation loss decreased (0.008218 --> 0.007408).  Saving model ...
Saving best model...
Saving best model...
Validation loss decreased (0.007408 --> 0.007222).  Saving model ...
Validation loss decreased (0.007222 --> 0.007203).  Saving model ...
Saving best model...
Saving best model...
EarlyStopping counter: 1 out of 7
Saving best model...
Saving best model...
Validation loss decreased (0.007203 --> 0.006922).  Saving model ...
Saving best model...
Saving best model...
Test Loss: 0.0065
Optimal Threshold: 0.2925
Best F1-Score at Optimal Thres

...

Saving best model...
Saving best model...
Saving best model...
Saving best model...
Validation loss decreased (inf --> 0.007664).  Saving model ...
Saving best model...
Validation loss decreased (0.007664 --> 0.007445).  Saving model ...
Validation loss decreased (0.007445 --> 0.006718).  Saving model ...
Saving best model...
Validation loss decreased (0.006718 --> 0.006626).  Saving model ...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Validation loss decreased (0.006626 --> 0.006074).  Saving model ...
Validation loss decreased (0.006074 --> 0.006068).  Saving model ...
Validation loss decreased (0.006068 --> 0.006067).  Saving model ...
Validation loss decreased (0.006067 --> 0.006008).  Saving model ...
Saving best model...
Validation loss decreased (0.006008 --> 0.005980).  Saving model ...
Validation loss decreased (0.005980 --> 0.005919).  Saving model ...
Saving best model...
Saving best model...
Validation loss decreased (0.005919 --> 0.

/ext3/miniforge3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Validation loss decreased (inf --> 0.007592).  Saving model ...
Saving best model...
Saving best model...
Validation loss decreased (0.007592 --> 0.007332).  Saving model ...
Validation loss decreased (0.007332 --> 0.006998).  Saving model ...
Saving best model...
EarlyStopping counter: 1 out of 7
Saving best model...
Validation loss decreased (0.006998 --> 0.006483).  Saving model ...
Saving best model...
Saving best model...
Saving best model...
Validation loss decreased (0.006483 --> 0.006269).  Saving model ...
Saving best model...
Saving best model...
Validation loss decreased (0.006269 --> 0.006071).  Saving model ...
EarlyStopping counter: 1 out of 7
Saving best model...
Saving best model...
EarlyStopping counter: 2 out of 7
Validation loss decreased (0.006071 --> 0.005954).  Saving model ...
EarlyStopping counter: 1 out of 7
EarlyStopping counter: 2 out 

/ext3/miniforge3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Validation loss decreased (inf --> 0.009410).  Saving model ...
Saving best model...
Validation loss decreased (0.009410 --> 0.008900).  Saving model ...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Saving best model...
Validation loss decreased (0.008900 --> 0.008124).  Saving model ...
Saving best model...
Saving best model...
EarlyStopping counter: 1 out of 7
Saving best model...
Saving best model...
Validation loss decreased (0.008124 --> 0.007769).  Saving model ...
EarlyStopping counter: 1 out of 7
Validation loss decreased (0.007769 --> 0.007769).  Saving model ...
EarlyStopping counter: 1 out of 7
Saving best model...
Validation loss decreased (0.007769 --> 0.007664).  Saving model ...
Validation loss decreased (0.007664 --> 0.007563).  Saving model ...
